In [ ]:
# import pandas as pd
# def tabela(df_protocolo):
#     return df_protocolo.head(10)

# # Criando o arquivo com codificação UTF-8
# with open('tabela_protocolo.py', 'w', encoding='utf-8') as f:
#     f.write("""
# import pandas as pd
# def tabela(df_protocolo):
#     return df_protocolo.head(10)
# """)


# 1 Importando as bibliotecas

In [ ]:

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os
from dotenv import load_dotenv
import requests
from urllib.parse import urljoin
from datetime import datetime, timedelta
import io
from io import BytesIO
import base64
from email.mime.image import MIMEImage
import openpyxl
from dateutil.relativedelta import relativedelta

# 2. Abrindo a sessão na polotrial

In [ ]:
# carregando os dados de acesso à api
load_dotenv()

api_username = os.getenv('API_USERNAME')
api_password = os.getenv('API_PASSWORD')
api_url = os.getenv("API_URL")

In [ ]:
# Corpo do login a ser utilizado no acesso
body = {
    "nome": api_username,
    "password":api_password
}

# Obtençao do token de acesso à polotrial
auth_url = urljoin(api_url, "/sessions")

response = requests.post(auth_url, json = body)

# Verificar a resposta
print(f"Status Code: {response.status_code}")
print(f"Headers: {response.headers}")
print(f"Content: {response.text}")

In [ ]:
# Extraindo o token
token = response.json()["token"]

# Incorporando a string Bearer para inserir
if token:
    auth_token = "Bearer " + token
    print(f"Auth Token: {auth_token}")
else:
    print("Falha ao obter o token.")

In [ ]:
url_request = "https://api.polotrial.com"

headers = {"Authorization": auth_token}

# 3. Funções globais

In [ ]:
# TODO: Carregar as variáveis de ambiente
load_dotenv(override=True)

api_username = os.getenv('API_USERNAME')
api_password = os.getenv('API_PASSWORD')
api_url = os.getenv("API_URL")

# TODO: Configurações do e-mail
enviar_para = os.getenv('DESTINATARIO')
print(f"Valor original do DESTINATARIO: {enviar_para}")
if enviar_para:
    enviar_para = [email.strip() for email in enviar_para.split(',')]
else:
    enviar_para = []
username_email = os.getenv('EMAIL_USERNAME')
password_email = os.getenv('EMAIL_PASSWORD')
server_email = os.getenv('EMAIL_SERVER')
port_email = int(os.getenv('EMAIL_PORT'))


# TODO: funções globais
# Obter o mês e o ano do mês atual menos 1 ano
mes_atual = datetime.now().month - 0
ano_atual = datetime.now().year
ano_anterior = datetime.now().year - 1
mes_anterior = datetime.now().month - 1
proximas_duas_semanas = (datetime.now()+timedelta(days=15)).strftime('%Y-%m-%d')
duas_ultimas_semanas = (datetime.now()-timedelta(days=15))
ultima_semana = (datetime.now() - timedelta(days=7))
proxima_semana = (datetime.now()+timedelta(days=7))
amanha = (datetime.now()+timedelta(days=0))
dois_ultimos_anos = (datetime.now()-timedelta(days=730)).date()

# TODO: Configuração CSS
css_hover = """
<style>
    tr:hover {
        background-color: #EC0E73 !important;
        color: #FFFFFF;
    }
    div {
        margin: 40px 0px 40px 0 px;
        overflow-x:auto;
    }
    h1 {
  font-size: 30px;
    }

    h2 {
    font-size: 20px;
    }

    p {
    font-size: 16px;
    }
    
    th {
        font-size: 16px;
        font-weight: bold;
        background-color: #041266;
        color: #FFFFFF;
        white-space: nowrap;
    }
    td {
        font-size: 14px 
    }
</style>

"""    

In [ ]:
# load_dotenv(override=True)
# enviar_para = os.getenv('DESTINATARIO')
# print(f"Valor original do DESTINATARIO: {enviar_para}")

# if enviar_para:
#     enviar_para = [email.strip() for email in enviar_para.split(',')]
# else:
#     enviar_para = []
# print(enviar_para)

In [ ]:
def extrair_ultima_informacao(x):
    if x is None:
        return None
    else:
        values_list = list(x.values())
        if len(values_list) == 0:
            return None
        else:
            return values_list[-1]

In [ ]:
mes_atual = datetime.now().month - 0
ano_atual = datetime.now().year
ano_anterior = datetime.now().year - 1
proximas_duas_semanas = (datetime.now()+timedelta(days=180)).strftime('%Y-%m-%d')
duas_ultimas_semanas = (datetime.now()-timedelta(days=15))
proxima_semana = (datetime.now()+timedelta(days=7)).strftime('%Y-%m-%d')
amanha = (datetime.now()+timedelta(days=0))

In [ ]:
pd.options.display.max_columns=None

In [ ]:
def extrair_ultima_informacao(x):
    if x is None:
        return None
    else:
        values_list = list(x.values())
        if len(values_list) == 0:
            return None
        else:
            return values_list[-1]


def extrair_segunda_informacao(x):
    if x is None:
        return None
    else:
        values_list = list(x.values())
        if len(values_list) == 0:
            return None
        else:
            return values_list[1]
        
def extrair_terceira_informacao(x):
    if x is None:
        return None
    else:
        values_list = list(x.values())
        if len(values_list) == 0:
            return None
        else:
            return values_list[2] if len(values_list) > 2 else None

        
def extrair_apelido_protocolo(x):
    if x is None:
        return None
    elif 'apelido_protocolo' in x:
        return x['apelido_protocolo']
    else:
        return None
    
    
def extrair_campo(x, *chaves):
# """
# Extrai um campo aninhado de um dicionário dado um conjunto de chaves.
# :param x: O dicionário de entrada.
# :param chaves: Chaves para navegar no dicionário.
# :return: O valor extraído ou None se a navegação falhar.
# """
    if isinstance(x, dict):
        for chave in chaves:
            x = x.get(chave)
            if x is None:
                return None
        return x
    return None

# 4. Protocolos

In [ ]:
rota_protocolo = url_request+"/protocolo?nested=true"
df_protocolo = requests.get(rota_protocolo, headers = headers).json()
df_protocolo = pd.DataFrame(df_protocolo)

In [ ]:
rota_generica = url_request+"/generica?nested=true"
df_generica = requests.get(rota_generica, headers = headers).json()
df_generica = pd.DataFrame(df_generica)

# Assinatura de TCLE

In [ ]:
rota_participantes = url_request+"/participantes?nested=true"
df_participantes = requests.get(rota_participantes, headers = headers).json()
df_participantes = pd.DataFrame(df_participantes)

In [ ]:

rota_visita_procedimentos = url_request+"/power_bi_participante_visita_procedimento"
df_visita_procedimentos = requests.get(rota_visita_procedimentos, headers = headers).json()
df_visita_procedimentos = pd.DataFrame(df_visita_procedimentos)


In [ ]:
rota_pagamento = url_request+"/pagamento"
df_pagamento = requests.get(rota_pagamento, headers = headers).json()
df_pagamento = pd.DataFrame(df_pagamento)

# Procedimentos Visita 

In [ ]:
# rota_participante_visita = url_request+"/participante_visita?nested=true"
# df_participante_visita = requests.get(rota_participante_visita, headers = headers).json()
# df_participante_visita = pd.DataFrame(df_participante_visita)

In [ ]:
rota_participante_visita = url_request+"/participante_visita?nested=true"
df_participante_visita = requests.get(rota_participante_visita, headers=headers).json()
df_participante_visita = pd.DataFrame(df_participante_visita)

# FLOWCHART

In [ ]:
rota_flowchart = url_request+"/protocolo_flowchart"
df_flowchart = requests.get(rota_flowchart, headers = headers).json()
df_flowchart = pd.DataFrame(df_flowchart)

# Protocolo Financeiro

In [ ]:
rota_protocolo_financeiro = url_request+"/protocolo_financeiro?nested=true"
df_protocolo_financeiro = requests.get(rota_protocolo_financeiro, headers = headers).json()
df_protocolo_financeiro = pd.DataFrame(df_protocolo_financeiro)

# Invoices/NF com Status Pendente ou Pago que estejam duplicadas

In [ ]:
rota_recebimento = api_url + "/recebimento?nested=true"
df_recebimento = requests.get(rota_recebimento, headers = headers).json()
df_recebimento = pd.DataFrame(df_recebimento)

In [ ]:
# dim_recebimentos = df_recebimento.copy()

In [ ]:
# colunas_a_extrair_recebimento=[
#     'dados_protocolo',
#     'dados_status_invoice',
#     'dados_moeda'
# ]

# for coluna in colunas_a_extrair_recebimento:
#     dim_recebimentos[coluna] = dim_recebimentos[coluna].apply(extrair_ultima_informacao)

In [ ]:
# dim_recebimentos=dim_recebimentos[[
#     'id',
#     'codigo_nota_fiscal',
#     'data_emissao',
#     'valor_enviado',
#     'data_prevista_pagamento',
#     'valor_recebido',
#     'data_recebimento',
#     'impostos_retidos_valor',
#     'dados_protocolo',
#     'dados_status_invoice',
#     'dados_moeda'
# ]]



In [ ]:
# # Nome do dataframe

# dataframe_recebimento = "Recebimentos"
# # Adicionar nova coluna com o valor para identificar o dataframe de origem


# dim_recebimentos['Origem'] = dataframe_recebimento






In [ ]:
# filtro = ['A emitir']
# dim_recebimentos=dim_recebimentos[~dim_recebimentos['codigo_nota_fiscal'].isin(filtro)]

In [ ]:
# dim_recebimentos['codigo_nota_fiscal'].duplicated().value_counts()

In [ ]:
# filtro = ['Cancelada']
# dim_recebimentos=dim_recebimentos[~dim_recebimentos['dados_status_invoice'].isin(filtro)]

In [ ]:
# valore_duplicados=dim_recebimentos['codigo_nota_fiscal'].duplicated(keep=False)
# invoices_duplicadas=dim_recebimentos[valore_duplicados]

In [ ]:
# invoices_duplicadas['data_emissao']= pd.to_datetime(invoices_duplicadas['data_emissao']).dt.tz_localize(None)

In [ ]:
# invoices_duplicadas=invoices_duplicadas.sort_values(by='codigo_nota_fiscal', ascending = True)

In [ ]:
# # Primeira período para titulo do email
# if not invoices_duplicadas.empty:
#     invoices_duplicadas_min = invoices_duplicadas['data_emissao'].min().strftime('%d/%m/%Y')
#     invoices_duplicadas_max = invoices_duplicadas['data_emissao'].max().strftime('%d/%m/%Y')
# else:
#     invoices_duplicadas_min = None
#     invoices_duplicadas_max = None

In [ ]:
# subject_email =(
#    f'Não existem invoices duplicadas'
#    if invoices_duplicadas_min==None
#    else f'A Invoice ou NF cadastrada em {invoices_duplicadas_min} possui duplicata'
#       if invoices_duplicadas_min == invoices_duplicadas_max
#       else f'As invoices ou Notas fiscais cadastradas entre {invoices_duplicadas_min} e {invoices_duplicadas_max} possuem duplicatas'
# )
# subject_email

In [ ]:
# # Salvando o DataFrame em um arquivo Excel
# def salvar_dataframe_como_excel(dataframe, filename='invoices_duplicadas.xlsx'):
#     buffer = BytesIO()
#     dataframe.to_excel(buffer, index=False, engine='openpyxl')
#     buffer.seek(0)
#     return buffer

In [ ]:
# # Função para criar a tabela do corpo do email 
# def filtrar_protocolo_invoices_duplicadas(dataframe, anos=3):
#     # Filtrar contratos com data de assinatura não nula
#     dataframe = dataframe.loc[dataframe['data_emissao'].notna(), :]
# # Verificar se o DataFrame filtrado está vazio
#     if invoices_duplicadas.empty:
#         return "Nenhum estudos cadastrado está sem a moeda"
#     else:
#         # Formatar o DataFrame para exibição em HTML
#         dataframe_filtrado = invoices_duplicadas.style\
#             .format(precision=3, thousands=".", decimal=',')\
#             .format_index(str.upper, axis=1)\
#             .set_properties(**{'background-color': 'white'}, **{'color': 'black'})\
#             .set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#EC0E73')]}])

#         return dataframe_filtrado.to_html(index=False)

# # Chamando a função
# invoices_duplicadas_html = filtrar_protocolo_invoices_duplicadas(invoices_duplicadas)

# # Função de envio do e-mail
# def enviar_email_protocolos_invoices_duplicadas():
#     try:
#         if invoices_duplicadas.empty:
#             print("Notas Fiscais e Invoices sem duplicatas")
#             return
        
#         # Criação do arquivo Excel em memória
#         excel_file = salvar_dataframe_como_excel(invoices_duplicadas)

#         msg = MIMEMultipart("related")
#         msg['From'] = username_email
#         msg['Bcc'] = ', '.join(enviar_para)
#         msg['Subject'] = subject_email
        
#         # Corpo do e-mail simplificado
#         body = f"""
#         <html>
#             <head>{css_hover}</head>
#             <body>
#                 <h2>{subject_email}</h2>
#                 <p>Gui, veja se assim está ok? o anexo está em .xlsx</p>
#                 <p>{invoices_duplicadas_html}</p>
#                 <p>Eu vim para te mandar mensagens, mua ha ha</p>
#             </body>
#         </html>
#         """
#         msg.attach(MIMEText(body, 'html'))
        
#         # Anexando o arquivo Excel
#         part = MIMEBase('application', 'octet-stream')
#         part.set_payload(excel_file.read())
#         encoders.encode_base64(part)
#         part.add_header(
#             'Content-Disposition',
#             f'attachment; filename="invoices_duplicadas.xlsx"'
#         )
#         msg.attach(part)
        
#         # Enviar o e-mail
#         with smtplib.SMTP(server_email, port_email) as server:
#             server.starttls()
#             server.login(username_email, password_email)
#             server.send_message(msg)
#         print(f"{subject_email}")
        
#     except Exception as e:
#         print(f"Erro ao enviar o e-mail: {e}")

# enviar_email_protocolos_invoices_duplicadas()

# Visitas realizadas e se elas estão vinculadas a alguma Invoice ou NF

In [ ]:
# fato_visitas_nf = df_participante_visita[[
#     'id',
#     'dados_participante',
#     'dados_visita',
#     'data_realizada',
#     'dados_status',
#     'dados_nota_fiscal'
#     ]].copy()

In [ ]:
# # Extrair 'co_protocolo'
# fato_visitas_nf['co_protocolo'] = fato_visitas_nf['dados_participante'].apply(
#     lambda x: extrair_campo(x, 'co_protocolo')
# )

# # Extrair 'id_participante'
# fato_visitas_nf['id_participante'] = fato_visitas_nf['dados_participante'].apply(
#     lambda x: extrair_campo(x, 'id_participante')
# )

# # Extrair 'apelido_protocolo'
# fato_visitas_nf['apelido_protocolo'] = fato_visitas_nf['dados_participante'].apply(
#     lambda x: extrair_campo(x, 'dados_protocolo', 'apelido_protocolo')
# )

# participante = ['dados_visita']

# for coluna in participante:
#     if coluna in fato_visitas_nf.columns:
#         fato_visitas_nf[coluna] = fato_visitas_nf[coluna].apply(
#             lambda x: extrair_segunda_informacao(x) if isinstance(x, dict) else None
#         )
#     else:
#         print(f"A coluna '{coluna}' não existe no DataFrame.")
        
# ultima_info=['dados_status','dados_nota_fiscal']

# for coluna in ultima_info:
#     if coluna in fato_visitas_nf.columns:
#         fato_visitas_nf.loc[:,coluna] = fato_visitas_nf[coluna].apply(extrair_ultima_informacao)
#     else:
#         print(f"A coluna '{coluna}' não existe no Dataframe")

In [ ]:
# fato_visitas_nf=fato_visitas_nf.drop('dados_participante', axis = 1)

In [ ]:
# colunas_data = ['data_realizada']

# # Converte cada coluna de data separadamente para melhorar o desempenho
# for coluna in colunas_data:
#     fato_visitas_nf[coluna] = pd.to_datetime(fato_visitas_nf[coluna], errors='coerce').dt.tz_localize(None).dt.date

In [ ]:
# fato_visitas_nf.rename(columns={
#     'id':'id_visita_nf',
#     'apelido_protocolo': 'Protocolo',
#     'dados_participante':'Participante',
#     'dados_visita':'Visita',
#     'data_realizada':'Data Realizada',
#     'dados_status':'Status da visita',
#     'dados_nota_fiscal':'Nota Fiscal'
#     }, inplace=True)

In [ ]:
# filtros = ['Realizada', 'Realizada parcialmente']

# fato_visitas_nf = fato_visitas_nf[
#     fato_visitas_nf['Status da visita'].isin(filtros)
# ]

In [ ]:
# fato_visitas_nf = fato_visitas_nf[fato_visitas_nf['Nota Fiscal'].isna()]

In [ ]:
# fato_visitas_nf.columns

In [ ]:
# fato_visitas_nf = fato_visitas_nf[['Protocolo',
#                                    'id_participante',
#                                    'Visita',
#                                    'Status da visita',
#                                    'Data Realizada',
#                                    'Nota Fiscal',
#                                    'co_protocolo',
#                                    ]]

In [ ]:
# colunas_data = ['Data Realizada']

# # Converte cada coluna de data separadamente para melhorar o desempenho
# for coluna in colunas_data:
#     fato_visitas_nf[coluna] = pd.to_datetime(fato_visitas_nf[coluna], errors='coerce').dt.tz_localize(None)

In [ ]:
# iniciativa = df_protocolo[['id', 'dados_tipo_de_iniciativa']]
# iniciativa.rename(columns={
#     'id':'co_protocolo',
#     'dados_tipo_de_iniciativa': 'Iniciatita'
#     }, inplace=True)

# colunas_a_extrair_recebimento=[
#     'Iniciatita'
# ]

# for coluna in colunas_a_extrair_recebimento:
#     iniciativa[coluna] = iniciativa[coluna].apply(extrair_ultima_informacao)

In [ ]:
# fato_visitas_nf = fato_visitas_nf.merge(iniciativa, how = 'left', on='co_protocolo')

In [ ]:
# tipo_de_estudo =['Patrocinador']
# fato_visitas_nf=fato_visitas_nf[fato_visitas_nf['Iniciatita'].isin(tipo_de_estudo)]

# filtro_visita=['Particularidades do Financeiro ']
# fato_visitas_nf=fato_visitas_nf[~fato_visitas_nf['Visita'].isin(filtro_visita)]

In [ ]:
# fato_visitas_nf = fato_visitas_nf[fato_visitas_nf['Visita'].notna()]

In [ ]:

# fato_visitas_nf = fato_visitas_nf[['id_participante',
#                                    'Protocolo',
#                                    'Iniciatita',
#                                    'Visita',
#                                    'Data Realizada',
#                                    'Status da visita',
#                                    'Nota Fiscal',
#                                    ]].sort_values(by='Data Realizada', ascending = True)

In [ ]:
# # Primeira período para titulo do email
# if not fato_visitas_nf.empty:
#     fato_visitas_nf_min = fato_visitas_nf['Data Realizada'].min().strftime('%d/%m/%Y')
#     fato_visitas_nf_max = fato_visitas_nf['Data Realizada'].max().strftime('%d/%m/%Y')
# else:
#     fato_visitas_nf_min = None
#     fato_visitas_nf_max = None

In [ ]:
# # Primeira período para titulo do email
# if not fato_visitas_nf.empty:
#     fato_visitas_nf_min = fato_visitas_nf['Data Realizada'].min().strftime('%d/%m/%Y')
#     fato_visitas_nf_max = fato_visitas_nf['Data Realizada'].max().strftime('%d/%m/%Y')
# else:
#     fato_visitas_nf_min = None
#     fato_visitas_nf_max = None

In [ ]:
# subject_email =(
#    f'Não existem sem nota fiscal ou invoice vinculada'
#    if fato_visitas_nf_min==None
#    else f'As visitas realizadas em {fato_visitas_nf_min} não possuem nota fiscal ou invoice vinculada'
#       if fato_visitas_nf_min == fato_visitas_nf_max
#       else f'As visitas realizadas entre {fato_visitas_nf_min} e {fato_visitas_nf_max} não possuem nota fiscal ou invoice vinculada'
# )
# subject_email

In [ ]:
# # Salvando o DataFrame em um arquivo Excel
# def salvar_dataframe_como_excel(dataframe, filename='visitas_sem_nf.xlsx'):
#     buffer = BytesIO()
#     dataframe.to_excel(buffer, index=False, engine='openpyxl')
#     buffer.seek(0)
#     return buffer

In [ ]:
# # Função para criar a tabela do corpo do email 
# def filtrar_protocolo_fato_visitas_nf(dataframe, anos=3):
#     # Filtrar contratos com data de assinatura não nula
#     dataframe = dataframe.loc[dataframe['Data Realizada'].notna(), :]
# # Verificar se o DataFrame filtrado está vazio
#     if fato_visitas_nf.empty:
#         return "Nenhuma visita realizada está sem a NF vinculada"
#     else:
#         # Formatar o DataFrame para exibição em HTML
#         dataframe_filtrado = fato_visitas_nf.style\
#             .format(precision=3, thousands=".", decimal=',')\
#             .format_index(str.upper, axis=1)\
#             .set_properties(**{'background-color': 'white'}, **{'color': 'black'})\
#             .set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#EC0E73')]}])

#         return dataframe_filtrado.to_html(index=False)

# # Chamando a função
# fato_visitas_nf_html = filtrar_protocolo_fato_visitas_nf(fato_visitas_nf)

# # Função de envio do e-mail
# def enviar_email_protocolos_fato_visitas_nf():
#     try:
#         if fato_visitas_nf.empty:
#             print("Todas as visitas estão com a NF ou Invoice vinculada")
#             return
        
#         # Criação do arquivo Excel em memória
#         excel_file = salvar_dataframe_como_excel(fato_visitas_nf)

#         msg = MIMEMultipart("related")
#         msg['From'] = username_email
#         msg['Bcc'] = ', '.join(enviar_para)
#         msg['Subject'] = subject_email
        
#         # Corpo do e-mail simplificado
#         body = f"""
#         <html>
#             <head>{css_hover}</head>
#             <body>
#                 <h2>{subject_email}</h2>
                
#                 <p>{fato_visitas_nf_html}</p>
#                 <p>Eu vim para te mandar mensagens, mua ha ha</p>
#             </body>
#         </html>
#         """
#         msg.attach(MIMEText(body, 'html'))
        
#         # Anexando o arquivo Excel
#         part = MIMEBase('application', 'octet-stream')
#         part.set_payload(excel_file.read())
#         encoders.encode_base64(part)
#         part.add_header(
#             'Content-Disposition',
#             f'attachment; filename="fato_visitas_nf.xlsx"'
#         )
#         msg.attach(part)
        
#         # Enviar o e-mail
#         with smtplib.SMTP(server_email, port_email) as server:
#             server.starttls()
#             server.login(username_email, password_email)
#             server.send_message(msg)
#         print(f"{subject_email}")
        
#     except Exception as e:
#         print(f"Erro ao enviar o e-mail: {e}")

# enviar_email_protocolos_fato_visitas_nf()

In [ ]:
# subject_email =(
#    f'Não existem sem nota fiscal ou invoice vinculada'
#    if fato_visitas_nf_min==None
#    else f'As visitas realizadas em {fato_visitas_nf_min} não possuem nota fiscal ou invoice vinculada'
#       if fato_visitas_nf_min == fato_visitas_nf_max
#       else f'As visitas realizadas entre {fato_visitas_nf_min} e {fato_visitas_nf_max} não possuem nota fiscal ou invoice vinculada'
# )
# subject_email

In [ ]:
# # Salvando o DataFrame em um arquivo Excel
# def salvar_dataframe_como_excel(dataframe, filename='visitas_sem_nf.xlsx'):
#     buffer = BytesIO()
#     dataframe.to_excel(buffer, index=False, engine='openpyxl')
#     buffer.seek(0)
#     return buffer

In [ ]:
# # Função para criar a tabela do corpo do email 
# def filtrar_protocolo_fato_visitas_nf(dataframe, anos=3):
#     # Filtrar contratos com data de assinatura não nula
#     dataframe = dataframe.loc[dataframe['Data Realizada'].notna(), :]
# # Verificar se o DataFrame filtrado está vazio
#     if fato_visitas_nf.empty:
#         return "Nenhuma visita realizada está sem a NF vinculada"
#     else:
#         # Formatar o DataFrame para exibição em HTML
#         dataframe_filtrado = fato_visitas_nf.style\
#             .format(precision=3, thousands=".", decimal=',')\
#             .format_index(str.upper, axis=1)\
#             .set_properties(**{'background-color': 'white'}, **{'color': 'black'})\
#             .set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#EC0E73')]}])

#         return dataframe_filtrado.to_html(index=False)

# # Chamando a função
# fato_visitas_nf_html = filtrar_protocolo_fato_visitas_nf(fato_visitas_nf)

# # Função de envio do e-mail
# def enviar_email_protocolos_fato_visitas_nf():
#     try:
#         if fato_visitas_nf.empty:
#             print("Todas as visitas estão com a NF ou Invoice vinculada")
#             return
        
#         # Criação do arquivo Excel em memória
#         excel_file = salvar_dataframe_como_excel(fato_visitas_nf)

#         msg = MIMEMultipart("related")
#         msg['From'] = username_email
#         msg['Bcc'] = ', '.join(enviar_para)
#         msg['Subject'] = subject_email
        
#         # Corpo do e-mail simplificado
#         body = f"""
#         <html>
#             <head>{css_hover}</head>
#             <body>
#                 <h2>{subject_email}</h2>
                
#                 <p>{fato_visitas_nf_html}</p>
#                 <p>Eu vim para te mandar mensagens, mua ha ha</p>
#             </body>
#         </html>
#         """
#         msg.attach(MIMEText(body, 'html'))
        
#         # Anexando o arquivo Excel
#         part = MIMEBase('application', 'octet-stream')
#         part.set_payload(excel_file.read())
#         encoders.encode_base64(part)
#         part.add_header(
#             'Content-Disposition',
#             f'attachment; filename="fato_visitas_nf.xlsx"'
#         )
#         msg.attach(part)
        
#         # Enviar o e-mail
#         with smtplib.SMTP(server_email, port_email) as server:
#             server.starttls()
#             server.login(username_email, password_email)
#             server.send_message(msg)
#         print(f"{subject_email}")
        
#     except Exception as e:
#         print(f"Erro ao enviar o e-mail: {e}")

# enviar_email_protocolos_fato_visitas_nf()

In [ ]:
# #acesso à base de dados
# dim_recebimentos = df_recebimento.copy()

# # Extraindo informações de dicionários
# colunas_a_extrair_recebimento=[
#     'dados_protocolo',
#     'dados_status_invoice',
#     'dados_moeda'
# ]

# for coluna in colunas_a_extrair_recebimento:
#     dim_recebimentos[coluna] = dim_recebimentos[coluna].apply(extrair_ultima_informacao)

# # dados de interesse    
# dim_recebimentos=dim_recebimentos[[
#     'id',
#     'codigo_nota_fiscal',
#     'data_emissao',
#     'valor_enviado',
#     'data_prevista_pagamento',
#     'valor_recebido',
#     'data_recebimento',
#     'impostos_retidos_valor',
#     'dados_protocolo',
#     'dados_status_invoice',
#     'dados_moeda'
# ]]

# # endereçando o dataframe
# dataframe_recebimento = "Recebimentos"
# dim_recebimentos['Origem'] = dataframe_recebimento

# # Primeiro Filtro
# filtro = ['A emitir']
# dim_recebimentos=dim_recebimentos[~dim_recebimentos['codigo_nota_fiscal'].isin(filtro)]

# # Segundo Filtro
# filtro2 = ['Cancelada']
# dim_recebimentos=dim_recebimentos[~dim_recebimentos['dados_status_invoice'].isin(filtro2)]

# # terceiro filtro
# filtro3 = ['Clínica Morumbi ']
# dim_recebimentos=dim_recebimentos[~dim_recebimentos['dados_protocolo'].isin(filtro3)]

# # Selecionando valores duplicados
# valore_duplicados=dim_recebimentos['codigo_nota_fiscal'].duplicated(keep=False)
# invoices_duplicadas=dim_recebimentos[valore_duplicados]

# # Tratameto da coluna de datas
# invoices_duplicadas['data_emissao']= pd.to_datetime(invoices_duplicadas['data_emissao']).dt.tz_localize(None)

# # Classificando os dados em ordem crescente
# invoices_duplicadas=invoices_duplicadas.sort_values(by='codigo_nota_fiscal', ascending = True)



# # Períodos para titulo do email
# if not invoices_duplicadas.empty:
#     invoices_duplicadas_min = invoices_duplicadas['data_emissao'].min().strftime('%d/%m/%Y')
#     invoices_duplicadas_max = invoices_duplicadas['data_emissao'].max().strftime('%d/%m/%Y')
# else:
#     invoices_duplicadas_min = None
#     invoices_duplicadas_max = None

# # Texto personalizado  
# subject_email =(
#    f'Não existem invoices duplicadas'
#    if invoices_duplicadas_min==None
#    else f'A Invoice ou NF cadastrada em {invoices_duplicadas_min} possui duplicata'
#       if invoices_duplicadas_min == invoices_duplicadas_max
#       else f'As invoices ou Notas fiscais cadastradas entre {invoices_duplicadas_min} e {invoices_duplicadas_max} possuem duplicatas'
# )

# # Salvando o DataFrame em um arquivo Excel
# def salvar_dataframe_como_excel(dataframe, filename='invoices_duplicadas.xlsx'):
#     buffer = BytesIO()
#     dataframe.to_excel(buffer, index=False, engine='openpyxl')
#     buffer.seek(0)
#     return buffer

# # Função para criar a tabela do corpo do email 
# def filtrar_protocolo_invoices_duplicadas(dataframe, anos=3):
#     # Filtrar contratos com data de assinatura não nula
#     dataframe = dataframe.loc[dataframe['data_emissao'].notna(), :]
# # Verificar se o DataFrame filtrado está vazio
#     if invoices_duplicadas.empty:
#         return "Nenhum estudos cadastrado está sem a moeda"
#     else:
#         # Formatar o DataFrame para exibição em HTML
#         dataframe_filtrado = invoices_duplicadas.style\
#             .format(precision=3, thousands=".", decimal=',')\
#             .format_index(str.upper, axis=1)\
#             .set_properties(**{'background-color': 'white'}, **{'color': 'black'})\
#             .set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#EC0E73')]}])

#         return dataframe_filtrado.to_html(index=False)

# # Chamando a função
# invoices_duplicadas_html = filtrar_protocolo_invoices_duplicadas(invoices_duplicadas)

# # Função de envio do e-mail
# def enviar_email_protocolos_invoices_duplicadas():
#     try:
#         if invoices_duplicadas.empty:
#             print("Notas Fiscais e Invoices sem duplicatas")
#             return
        
#         # Criação do arquivo Excel em memória
#         excel_file = salvar_dataframe_como_excel(invoices_duplicadas)

#         msg = MIMEMultipart("related")
#         msg['From'] = username_email
#         msg['Bcc'] = ', '.join(enviar_para)
#         msg['Subject'] = subject_email
        
#         # Corpo do e-mail simplificado
#         body = f"""
#         <html>
#             <head>{css_hover}</head>
#             <body>
#                 <h2>{subject_email}</h2>
#                 <p>{invoices_duplicadas_html}</p>
#                 <p>Eu vim para te mandar mensagens, mua ha ha</p>
#             </body>
#         </html>
#         """
#         msg.attach(MIMEText(body, 'html'))
        
#         # Anexando o arquivo Excel
#         part = MIMEBase('application', 'octet-stream')
#         part.set_payload(excel_file.read())
#         encoders.encode_base64(part)
#         part.add_header(
#             'Content-Disposition',
#             f'attachment; filename="invoices_duplicadas.xlsx"'
#         )
#         msg.attach(part)
        
#         # Enviar o e-mail
#         with smtplib.SMTP(server_email, port_email) as server:
#             server.starttls()
#             server.login(username_email, password_email)
#             server.send_message(msg)
#         print(f"{subject_email}")
        
#     except Exception as e:
#         print(f"Erro ao enviar o e-mail: {e}")

# enviar_email_protocolos_invoices_duplicadas()


# Procedimentos duplicados

In [234]:
dim_visita_procedimentos = df_visita_procedimentos.copy()

In [235]:
dim_visita_procedimentos.shape

(71837, 78)

In [236]:
procedimentos_extras=dim_visita_procedimentos[[
    'dados_protocolo_procedimento.co_protocolo',
    'dados_participante_visita.co_participante',
    'dados_participante_visita.nome_tarefa',
    'data_executada',
    'opcional',
    'dados_protocolo_procedimento.nome_procedimento_estudo',
    'dados_participante_visita_procedimento_executor.dados_pessoa_executor.ds_nome',
    'dados_visita_procedimento.opcional',
    'dados_visita_procedimento.disponibilidade',
    'co_visita_procedimento',
    'dados_nota_fiscal.codigo_nota_fiscal'


]]
procedimentos_extras=dim_visita_procedimentos.copy()
procedimentos_extras.rename(columns={
    'dados_protocolo_procedimento.co_protocolo': 'co_protocolo',
    'dados_participante_visita.co_participante':'co_participante',
    'dados_participante_visita.nome_tarefa': 'Visita',
    'data_executada':'Data Executada',
    'opcional': 'Tipo de procedimento' ,
    'dados_protocolo_procedimento.nome_procedimento_estudo': 'Procedimento',
    'dados_participante_visita_procedimento_executor.dados_pessoa_executor.ds_nome':'Executor',
    'dados_visita_procedimento.opcional': 'Opcional 2',
    'dados_visita_procedimento.disponibilidade': 'Disponibilidade',
    'co_visita_procedimento': 'Codigo Procedimentos',
    'dados_nota_fiscal.codigo_nota_fiscal':'codigo_nota_fiscal'
    }, inplace=True)

In [237]:
protocolo = df_protocolo[[
    'id',
    'apelido_protocolo',
    'dados_co_centro',
    'dados_tipo_de_iniciativa'
    ]].copy()


extrair = ['dados_co_centro','dados_tipo_de_iniciativa']
for coluna in extrair:
    protocolo[coluna] = protocolo[coluna].apply(extrair_ultima_informacao)
    
protocolo.rename(columns={
    'id':'co_protocolo',
    'apelido_protocolo': 'Protocolo',
    'dados_co_centro':'Centro',
    'dados_tipo_de_iniciativa': 'Iniciativa'
    }, inplace=True)

In [238]:
procedimentos_extras = procedimentos_extras.merge(protocolo, how = 'left', on='co_protocolo')

In [239]:
participantes = df_participantes[[
    'id',
    'id_participante',
    'dados_status',
    'Braco'
    ]].copy()

extrair2 = ['dados_status','Braco']
for coluna in extrair2:
    participantes[coluna] = participantes[coluna].apply(extrair_ultima_informacao)
    
participantes.rename(columns={
    'id':'co_participante',
    'id_participante': 'Participante',
    'dados_status': 'Status do participante',
    'Braco': 'Braço no estudo'
    }, inplace=True)

procedimentos_extras = procedimentos_extras.merge(participantes, how = 'left', on='co_participante')

In [ ]:
# procedimentos_extras['Tipos de procedimento'] = procedimentos_extras['Codigo Procedimentos'].fillna('Extra')
# procedimentos_extras.loc[procedimentos_extras['Disponibilidade']==2.0,'Tipos de procedimento' ] = 'Condicional'
# # Ajustando para "null" qualquer valor diferente de "extra" ou "condicional"
# procedimentos_extras['Tipos de procedimento'] = procedimentos_extras['Tipos de procedimento'].apply(lambda x: x if x in ['Extra', 'Condicional'] else np.nan)

In [ ]:
# procedimentos_extras=procedimentos_extras[(procedimentos_extras['Tipos de procedimento']=='Extra')|(procedimentos_extras['Tipos de procedimento']=='Condicional')]

In [224]:
procedimentos_extras=procedimentos_extras[[
    'Protocolo','Iniciativa', 'Centro', 'Participante', 'Status do participante', 'Visita','Braço no estudo','Data Executada', 'Procedimento', 'Executor'
]]

In [225]:
colunas_data = ['Data Executada']

# Converte cada coluna de data separadamente para melhorar o desempenho
for coluna in colunas_data:
    procedimentos_extras[coluna] = pd.to_datetime(procedimentos_extras[coluna], errors='coerce').dt.tz_localize(None)

In [226]:
filtro3 = ['Patrocinador']
procedimentos_extras=procedimentos_extras[procedimentos_extras['Iniciativa'].isin(filtro3)]

In [214]:
procedimentos_extras.columns

Index(['Protocolo', 'Iniciativa', 'Centro', 'Participante',
       'Status do participante', 'Visita', 'Braço no estudo', 'Data Executada',
       'Procedimento', 'Executor'],
      dtype='object')

In [227]:
procedimentos_extras = procedimentos_extras[procedimentos_extras.duplicated(subset =['Protocolo', 'Centro', 'Participante','Visita','Data Executada','Procedimento'], keep=False)]

In [ ]:
# procedimentos_extras=procedimentos_extras[(procedimentos_extras['Tipos de procedimento'].str.contains("Condicional", regex=False, na=False))|(procedimentos_extras['Tipos de procedimento'].str.contains("Extra", regex=False, na=False))].sort_values(by='Data Executada', ascending = True)

In [228]:
# # Filtrar os EA/EAS que ocorreram nos últimos 7 dias
procedimentos_duplicados = procedimentos_extras[
    (procedimentos_extras['Data Executada'].dt.year == ano_atual)&
    (procedimentos_extras['Data Executada'].dt.month <= mes_atual) #Após a primeira leva, alterar para delay de 1 semana
]

In [229]:
# Primeira período para titulo do email
if not procedimentos_duplicados.empty:
    procedimentos_duplicados_min = procedimentos_duplicados['Data Executada'].min().strftime('%d/%m/%Y')
    procedimentos_duplicados_max = procedimentos_duplicados['Data Executada'].max().strftime('%d/%m/%Y')
else:
    procedimentos_duplicados_min = None
    procedimentos_duplicados_max = None

In [231]:
subject_email =(
   f'Não existem estudos com procedimentos duplicados'
   if procedimentos_duplicados_min==None
   else f'As visitas realizadas em {procedimentos_duplicados_min} não possuem procedimentos duplicados'
      if procedimentos_duplicados_min == procedimentos_duplicados_max
      else f'As visitas realizadas entre {procedimentos_duplicados_min} e {procedimentos_duplicados_max} apresentam procedimentos duplicados'
)
subject_email

'As visitas realizadas entre 23/01/2024 e 16/09/2024 apresentam procedimentos duplicados'

In [232]:
# Salvando o DataFrame em um arquivo Excel
def salvar_dataframe_como_excel(dataframe, filename='procedimentos_duplicados.xlsx'):
    buffer = BytesIO()
    dataframe.to_excel(buffer, index=False, engine='openpyxl')
    buffer.seek(0)
    return buffer

In [233]:
# Função para criar a tabela do corpo do email 
def filtrar_protocolo_procedimentos_duplicados(dataframe, anos=3):
    # Filtrar contratos com data de assinatura não nula
    dataframe = dataframe.loc[dataframe['Data Executada'].notna(), :]
# Verificar se o DataFrame filtrado está vazio
    if procedimentos_duplicados.empty:
        return "Sem procedimentos Condicionais e/ou Extras"
    else:
        # Formatar o DataFrame para exibição em HTML
        dataframe_filtrado = procedimentos_duplicados.style\
            .format(precision=3, thousands=".", decimal=',')\
            .format_index(str.upper, axis=1)\
            .set_properties(**{'background-color': 'white'}, **{'color': 'black'})\
            .set_table_styles([{'selector': 'td:hover', 'props': [('background-color', '#EC0E73')]}])

        return dataframe_filtrado.to_html(index=False)

# Chamando a função
procedimentos_duplicados_html = filtrar_protocolo_procedimentos_duplicados(procedimentos_duplicados)

# Função de envio do e-mail
def enviar_email_protocolos_procedimentos_duplicados():
    try:
        if procedimentos_duplicados.empty:
            print("Não foram detectados procedimentos condicionais e/ou extras")
            return
        
        # Criação do arquivo Excel em memória
        excel_file = salvar_dataframe_como_excel(procedimentos_duplicados)

        msg = MIMEMultipart("related")
        msg['From'] = username_email
        msg['Bcc'] = ', '.join(enviar_para)
        msg['Subject'] = subject_email
        
        # Corpo do e-mail simplificado
        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <h2>{subject_email}</h2>
                
                <p>{procedimentos_duplicados_html}</p>
                <p>Eu vim para te mandar mensagens, mua ha ha</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))
        
        # Anexando o arquivo Excel
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(excel_file.read())
        encoders.encode_base64(part)
        part.add_header(
            'Content-Disposition',
            f'attachment; filename="procedimentos_duplicados.xlsx"'
        )
        msg.attach(part)
        
        # Enviar o e-mail
        with smtplib.SMTP(server_email, port_email) as server:
            server.starttls()
            server.login(username_email, password_email)
            server.send_message(msg)
        print(f"{subject_email}")
        
    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

enviar_email_protocolos_procedimentos_duplicados()

As visitas realizadas entre 23/01/2024 e 16/09/2024 apresentam procedimentos duplicados
